# PyCitySchools
This challenge creates dataframes by loading csv files containing data about students for a school district. By using Pandas, the data is cleaned up, summarized and analysed to report on the districts performance.

### Raw Data
The district has to csv files containing summary data. The first one contains school information such as school type, size (number of students), and budget. The second file contains data for each student in the district including their ID, name, gender, school as well as their math and reading scores. 

In order to us pandas and perform the necessary analysis, the csv files are converted into dataframes. 

In [1]:
# Get imports
import pandas as pd
import os
import math

# Set path names
schools_path = os.path.join('Resources','schools_complete.csv')
students_path = os.path.join('Resources','students_complete.csv')

# Create dataframes
schools_df = pd.read_csv(schools_path, delimiter=',')
students_df = pd.read_csv(students_path, delimiter=',')

# Combine both into single dataframe and see what we're working with
students_full_df = pd.merge(students_df, schools_df, how='left', on=['school_name'])
students_full_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


By using a simple check, it is found the data is complete and not containing any missing numbers or entries.

In [2]:
# df.count shows non NA values for each column
students_full_df.count()

Student ID       39170
student_name     39170
gender           39170
grade            39170
school_name      39170
reading_score    39170
math_score       39170
School ID        39170
type             39170
size             39170
budget           39170
dtype: int64

In [3]:
# Check the data types
students_full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39170 entries, 0 to 39169
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Student ID     39170 non-null  int64 
 1   student_name   39170 non-null  object
 2   gender         39170 non-null  object
 3   grade          39170 non-null  object
 4   school_name    39170 non-null  object
 5   reading_score  39170 non-null  int64 
 6   math_score     39170 non-null  int64 
 7   School ID      39170 non-null  int64 
 8   type           39170 non-null  object
 9   size           39170 non-null  int64 
 10  budget         39170 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 3.6+ MB


### Arranging Data
The data provided does not include whether students area passing each subject. A grade of 70 or higher is used here as benchmarch for students passing math and reading however this number could be changed to explore the data further. Passing overall is added to indicate a student passing both math and reading.

In [4]:
# Create variable to control passing grade - default is higher than 70
passing = 69.99999

# Create boolean columns for passing math, reading, and total (passing both) with set to False by default
students_full_df['passing_math'] = False
students_full_df['passing_reading'] = False
students_full_df['passing_overall'] = False

# Use bitwise if logic to swhitch passing to True for students above passing
students_full_df.loc[students_full_df['math_score'] > passing, 'passing_math'] = True
students_full_df.loc[students_full_df['reading_score'] > passing, 'passing_reading'] = True
students_full_df.loc[(students_full_df['passing_math'] == True) & (students_full_df['passing_reading'] == True), 'passing_overall'] = True

# Take a look at df with new columns
students_full_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,passing_math,passing_reading,passing_overall
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,True,False,False
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,False,True,False
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,False,True,False
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,False,False,False
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,True,True,True


For this report the district is primarily concerned with performance at the school level. The data is given for each student so Pandas is used to group it by school and clean the data up further.

In [5]:
# Create a new df from grouping by school name and summing over values 
school_summary_df = students_full_df.groupby(['school_name','type']).sum().reset_index()

# Normalize size using sqrt function
school_summary_df['size'] = school_summary_df['size'].apply(lambda x: math.sqrt(x))

# Normalize budget by dividing by size
school_summary_df['budget'] = school_summary_df['budget']/school_summary_df['size']

# Create per student budget by dividing by size again
school_summary_df['Per Student Budget'] = school_summary_df['budget']/school_summary_df['size']

# Normalize math and reading score by dividing by size
school_summary_df['math_score'] = school_summary_df['math_score']/school_summary_df['size']
school_summary_df['reading_score'] = school_summary_df['reading_score']/school_summary_df['size']

# Covert math, reading and overall passings to percents by divding by size and multiplying by 100
school_summary_df['passing_math'] = 100 * school_summary_df['passing_math']/school_summary_df['size']
school_summary_df['passing_reading'] = 100 * school_summary_df['passing_reading']/school_summary_df['size']
school_summary_df['passing_overall'] = 100 * school_summary_df['passing_overall']/school_summary_df['size']
school_summary_df.head()

,school_name,type,Student ID,reading_score,math_score,School ID,size,budget,passing_math,passing_reading,passing_overall,Per Student Budget
0,Bailey High School,District,101303896,81.033963,77.048432,34832,4976.0,3124928.0,66.680064,81.933280,54.642283,628.0
1,Cabrera High School,Charter,31477307,83.975780,83.061895,11148,1858.0,1081356.0,94.133477,97.039828,91.334769,582.0
2,Figueroa High School,District,12949059,81.158020,76.711767,2949,2949.0,1884411.0,65.988471,80.739234,53.204476,639.0
3,Ford High School,District,99055935,80.746258,77.102592,35607,2739.0,1763916.0,68.309602,79.299014,54.289887,644.0
4,Griffin High School,Charter,19077394,83.816757,83.351499,5872,1468.0,917500.0,93.392371,97.138965,90.599455,625.0


### Disrtict Summary
A district summary can be used as an overall average to measure each schools performance

In [6]:
# Create a new df to summarize district level data
district_report_df = pd.DataFrame()

# Number of schools is the length of the school summary df
district_report_df.loc[0, 'Total Schools'] = '{:.0f}'.format(len(school_summary_df['school_name']))

# Number of students is sum of each schools size
district_report_df.loc[0, 'Total Students'] = school_summary_df['size'].sum()

# Total budget is the sum of each schools budget
district_report_df.loc[0, 'Total Budget'] = '${:,.0f}'.format(school_summary_df['budget'].sum())

# district average is school size * schoo average / total size
district_report_df['Average Math Score'] = '{:.2f}'.format((school_summary_df['math_score']*school_summary_df['size']).sum()/district_report_df.iloc[0,1])
district_report_df['Average Reading Score'] = '{:.2f}'.format((school_summary_df['reading_score']*school_summary_df['size']).sum()/district_report_df.iloc[0,1])
district_report_df['% Passing Math'] = '{:.2f}'.format((school_summary_df['passing_math']*school_summary_df['size']).sum()/district_report_df.iloc[0,1])
district_report_df['% Passing Reading'] = '{:.2f}'.format((school_summary_df['passing_reading']*school_summary_df['size']).sum()/district_report_df.iloc[0,1])
district_report_df['% Passing Reading'] = '{:.2f}'.format((school_summary_df['passing_overall']*school_summary_df['size']).sum()/district_report_df.iloc[0,1])

# Use map to change formating of toatl students 
district_report_df.loc[0,'Total Students'] = '{:,.0f}'.format(district_report_df.loc[0,'Total Students'])

district_report_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading
0,15,"39,170","$24,649,428",78.99,81.88,74.98,65.17


### School Summary
The school summary shows performance datafor each school including average math and reading scores and % passing for math, reading and overall. 

In [7]:
# Create school level df and apply formating
school_report_df = pd.DataFrame()
school_report_df['School Name'] = school_summary_df['school_name']
school_report_df['School Type'] = school_summary_df['type']
school_report_df['Total Students'] = school_summary_df['size'].map('{:,.0f}'.format)
school_report_df['Total School budget'] = school_summary_df['budget'].map('${:,.0f}'.format)
school_report_df['Per Student Budget'] = school_summary_df['Per Student Budget'].map('${:,.0f}'.format)
school_report_df['Average Math Score'] = school_summary_df['math_score'].map('{:,.2f}'.format)
school_report_df['Average Reading Score'] = school_summary_df['reading_score'].map('{:,.2f}'.format)
school_report_df['% Passing Math'] = school_summary_df['passing_math'].map('{:,.2f}'.format)
school_report_df['% Passing Reading'] = school_summary_df['passing_reading'].map('{:,.2f}'.format)
school_report_df['% Passing Overall'] = school_summary_df['passing_overall'].map('{:,.2f}'.format)
school_report_df

,School Name,School Type,Total Students,Total School budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,Bailey High School,District,"4,976","$3,124,928",$628,77.05,81.03,66.68,81.93,54.64
1,Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13,97.04,91.33
2,Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99,80.74,53.20
3,Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31,79.30,54.29
4,Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39,97.14,90.60
5,Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75,80.86,53.53
6,Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51,96.25,89.23
7,Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68,81.32,53.51
8,Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06,81.22,53.54
9,Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59,95.95,90.54


### Top Perfomoring Schools by % of Students Passing Overall
A good measure of the schools performance would be % of students passing overall since the schools' objectives should include getting students to score well in both math and reading. This is shown by using Pandas to sort the datframe and display the top results.

In [8]:
# Sort school summary df in descending order
top_school_df = school_summary_df.sort_values('passing_overall', ascending=False, ignore_index=True)

# Create report df with correct formating
top_report_df = pd.DataFrame()
top_report_df['School Name'] = top_school_df['school_name']
top_report_df['School Type'] = top_school_df['type']
top_report_df['Total Students'] = top_school_df['size'].map('{:,.0f}'.format)
top_report_df['Total School budget'] = top_school_df['budget'].map('${:,.0f}'.format)
top_report_df['Per Student Budget'] = top_school_df['Per Student Budget'].map('${:.0f}'.format)
top_report_df['Average Math Score'] = top_school_df['math_score'].map('{:,.2f}'.format)
top_report_df['Average Reading Score'] = top_school_df['reading_score'].map('{:,.2f}'.format)
top_report_df['% Passing Math'] = top_school_df['passing_math'].map('{:,.2f}'.format)
top_report_df['% Passing Reading'] = top_school_df['passing_reading'].map('{:,.2f}'.format)
top_report_df['% Passing Overall'] = top_school_df['passing_overall'].map('{:,.2f}'.format)

# Show the top five results
top_report_df.head(5)

,School Name,School Type,Total Students,Total School budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13,97.04,91.33
1,Thomas High School,Charter,"1,635","$1,043,130",$638,83.42,83.85,93.27,97.31,90.95
2,Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39,97.14,90.60
3,Wilson High School,Charter,"2,283","$1,319,574",$578,83.27,83.99,93.87,96.54,90.58
4,Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59,95.95,90.54


### Bottom Performing Schools by % of Students Passing Overall
Similarly, the bottom performing schools is bottom results of the sorted dataframe.

In [9]:
# Show the bottom five results
top_report_df.iloc[[14,13,12,11,10],:]

,School Name,School Type,Total Students,Total School budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
14,Rodriguez High School,District,"3,999","$2,547,363",$637,76.84,80.74,66.37,80.22,52.99
13,Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99,80.74,53.20
12,Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68,81.32,53.51
11,Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75,80.86,53.53
10,Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06,81.22,53.54


### Math Scores by Grade Level
It may also be insightful for the district to look at trends across grades - specifically math scores for each grade across the district. This is also achieved by using conditional filtering.

In [44]:
# Intialize dfs to store results by grade
grade_by_school_for_math = pd.DataFrame()
grade_by_school_for_reading = pd.DataFrame()

# Create to lists to iterate over
grades = ['9th', '10th', '11th', '12th']
subjects = ['math_score', 'reading_score']

# double for loop to iterate through each grade, subject pair
for grade in grades:
    subject_by_school_for_grade = pd.DataFrame({'math_score':[],'reading_score':[]})
    for subject in subjects:
        # Create a place holder df that stores math_score, reading_score and school_name for every row with current grade level
        subject_and_school_for_grade = pd.DataFrame()
        # Store scores for each subject in temp df
        subject_and_school_for_grade[[subject,'school_name']] = students_full_df.loc[(students_full_df['grade'] == grade), [subject,'school_name']]
        subject_and_school_for_grade.rename(columns={'school_name':'School Name'}, inplace=True)
        subject_by_school_for_grade[subject] = subject_and_school_for_grade.groupby('School Name').mean()
    grade_by_school_for_math[grade] = subject_by_school_for_grade['math_score']
    grade_by_school_for_math[grade] = grade_by_school_for_math[grade].map('{:.02f}'.format)
    grade_by_school_for_reading[grade] = subject_by_school_for_grade['reading_score']
    grade_by_school_for_reading[grade] = grade_by_school_for_reading[grade].map('{:.2f}'.format)
    
grade_by_school_for_math

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


### Reading Scores by Grade Level
The reading scores by grade level are obtained in the same way.

In [45]:
grade_by_school_for_reading

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


### Scores by School Spending

In [93]:
# sort raw df by schools
scores_budget_by_schools =  students_full_df.groupby(['school_name','type']).mean().reset_index()

# Create budgett per student column
scores_budget_by_schools['Budget per Student'] = scores_budget_by_schools['budget'] / scores_budget_by_schools['size']

# Intialize bins (these can be adjusted for analysis)
bins = {'<$585': 0, '$585-$630': 585, '$630-$645': 630, '$645-$680': 645, '': 680}

# Initialize df that stores scores sorted by budget bins
scores_by_budgets = pd.DataFrame()

# For loop over bin values
for i in range(4):
    scores_by_budgets[list(bins.keys())[i]] = scores_budget_by_schools.loc[(scores_budget_by_schools['Budget per Student'] >= list(bins.values())[i]) & (scores_budget_by_schools['Budget per Student'] < list(bins.values())[i + 1]), ]
    #print(list(bins.keys())[i])

ValueError: Expected a 1D array, got an array with shape (4, 12)